## 4. 데이터 분할

## 4.1 단순 데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split

# 종속 변수와 독립변수가 분리되어 있는 경우
X_train, X_테스트(검증), y_train, y_테스트(검증) = train_test_split(독립변수, 종속변수, test_size=, random_state=, shuffle=, stratify=종속변수)

# 종속 변수가 한 데이터프레임에 있는 경우
df_train, df_테스트(혹은 검증) = train_test_split(데이터, test_size= , random_state=, shuffle=, stratify=)

X_train = df_train[Y제외]
Y_train = df_train[Y선택]
X_valid = df_valid[Y제외]
Y_valid = df_valid[Y선택]

- stratify를 적용하면 종속변수의 원래 비율대로 split, train_test_split에서는 기본값이 False
- cross_validation을 하려면 
    - 원래 데이터를 train, test로 나누고 다시 train을 가지고 cross validation
    - 원본 -> train, test
    - train -> cross_validation(train, valid)

# 5. Model

- 데이터가 매우 크고 고차원이면 Tree 기반 모델 대신 선형 모형을 사용해야 하는가?
    - RandomForest는 sparse matrix 데이터에서 잘 작동하지 않음
    - 속도와 메모리 사용에 제약이 있는 경우
        - 선형모델이 더 적합(파이썬 라이브러리를 활용한 머신러닝 p.121)

In [1]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV, learning_curve

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC


## 5.1 Classification

## 5.1.1 Support Vector Machine(linear)

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='linear', C=1e10, gamma=0.1, probability=True)
model.fit(훈련_X, 훈련_Y)

# C : 슬랙 변수 크게하면 넘어가는 데이터를 없게 하고 작게 하면 마진을 최대화
# probability : predict_proba를 반환하느냐 안하느냐

- 속성값
- n_support_ : 각 클래스의 서포트 개수
- support_vectors_ : 각 클래스의 서포트의 x값. x+ 와 x-
- coef_ : w벡터
- intercept_ : -w0
- dual_coef_ : 각 원소가 a_i * y_i 로 이루어진 벡터(-a와 +a를 의미)
               더하면 0됨 ∑ai*yi = 0 (w0으로 미분값)

## 5.1.2 Radial Support Vector Machine

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='rbf', C=, gamma=, probability=True)
model.fit(훈련_X, 훈련_Y)

# kernel : poly, rbf, sigmoid ... 
# gamma : 크면 과적합

## 5.1.3 KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors= )

## 5.1.4 DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

트리 = DecisionTreeClassifier(criterion='entropy or gini', max_depth=, random_state=)
트리.fit(훈련_X, 훈련_Y)

- random_state: feature를 고른 후 각 threshold 별로 정보획득량을 계산하는데 모든 구간을 다 잘라보는게 아니고 랜덤으로 골라서 나눠본다


- **사전 가지치기(보통 1개만 지정해도 충분)**
    - max_depth: 최대 몇번 나눌 것인가
    - max_leaf_nodes: 리프 노드의 최대 개수
    - min_sample_split: 노드를 분할하기 위한 최소한의 샘플 데이터 수
    - min_samples_leaf: 리프 노드가 되기 위한 최소한의 샘플 데이터 개수
        - 비대칭적 데이터의 경우 특정 클래스의 데이터가 매우 작을 수 있으므로 이런 경우 작게 설정
    - min_impurity_decrease: 분할로 얻어질 불순도 감소 최솟값


- **사후 가지치기**
    - ccp_alpha: 비용 복잡도 기반(파이썬 라이브러리를 활용한 머신러닝 p.106)


- DecisionTree기반 모델은 데이터 스케일의 영향을 받지 않으므로 특성의 정규화나 표준화같은 전처리 과정이 필요없음
- 이진 특성과 연속적인 특성이 혼합되어 있을때도 잘 작동
- Greedy 문제가 있음

#### 개별 모델 하이퍼 파라미터에 따른 성능 시각화

In [ ]:
from sklearn import metrics

parameter_index = list(range(최소, 최대))
scores = pd.Series()

for i in parameter_index:
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(훈련_X, 훈련_Y)
    prediction = model.predict(검증_X)
    scores = scores.append(pd.Series(metrics.accuracy_score(prediction, 검증_Y)))  # 여기서는 score로 accuracy 사용

plt.subplots(figsize=(12, 6))
plt.plot(parameter_index, scores)
plt.xticks(parameter_index)
plt.show()
print('The max value is', scores.max())
print('The best parameter is', parameter_index[np.argmax(a)])

- accuracy : metrics.accuracy_score
- precision : metrics.precision_score
- recall : metrics.recall_score
- roc_auc : metrics.roc_auc_score
- f1 : metrics.f1_score

#### predict_proba에서 임계점 찾기

In [ ]:
score_array = []
for temp_thres in np.arange(0, 1, 0.01):
    threshold = temp_thres
    prediction1 = (model.predict_proba(검증_X)[:, 1] > temp_thres).astype(int)
    temp_score = 100 * metrics.accuracy_score(prediction1, 검증_Y)
    print('Accuracy for rbf SVM is {:.2f}%', 100 * metrics.accuracy_score(prediction1, 검증_Y))
    score_array.append(temp_score)
    
# 시각화
plt.plot(np.arange(0, 1, 0.01), score_array)
plt.xlabel('threshold')
plt.ylabel('accuracy score')

# 가장 높은 accuracy_score에서의 threshold
print(argmax(score_array))

## 5.2 Regression

# 6. Ensembling

## 6.1 Classification

## 6.1.1 Voting Classifier
- 분류 문제에서만 가능하다
- 회귀문제는 여러 모델을 동시에 쓰기 힘듦
- 성능이 0.5 이하의 모델을 사용하면 더욱 성능이 나빠진다

In [ ]:
from sklearn.ensemble import VotingClassifier

model1 = 분류기1(파라미터)
model2 = 분류기2(파라미터)
model3 = 분류기3(파라미터)

voting_model = VotingClassifier(estimators=[('이름1', model1),
                                            ('이름2', model2),
                                            ('이름3', model3)],
                               voting='소프트 or 하드', weights=[비율1, 비율2, 비율3], n_jobs=-1)
voting_model.fit(훈련_X, 훈련_Y)

## 6.1.2 Bagging
- 분산이 높은 모델에서 효과가 좋음

### 6.1.2.1 일반적인 Bagging

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier(base모델, n_estimators=, bootstrap=, max_samples=, max_features=, random_state=, n_jobs=-1)
model.fit(훈련_X, 훈련_Y)

- n_estimators: 모형 갯수
- bootstrap: 데이터의 중복 사용 여부. 디폴트 True
- max_samples: 선택할 샘플의 수 혹은 비율 디폴트 1.0
- bootstrap_features: 특징 차원 중복 디폴트 False
- max_features: 독립 변수중  선택할 차원의 수 혹은 비율 디폴트 1.0

- 분류기가 predict_proba()를 지원하는 경우 확률값을 평균하여 예측

### 6.1.2.2 RandomForest
- DecisionTree에 기반한 Bagging, 부트스트랩 샘플링도 이루어진다
- 각 노드마다 일부의 feature를 주고 그 안에서 가장 정보획득이 높은 Feature 선택
- 분기 후에 데이터를 더 주지 않는다(지도학습-47. 25분)
- 결과는 출력 레이블의 확률로 soft voting방식

In [ ]:
from sklearn.ensemble import RandomForestClassifier

모델 = RandomForestClassifier(n_estimators=, max_features=, max_depth=, random_state=, n_jobs=-1)

- max_features: 기본값 'auto', 분류에서는 sqrt(n_features), 회귀에서는 n_features
- 다른 매개변수 DecisionTree와 같음

### 6.1.2.3 Extremely Randomized Trees
- 각 노드에서 분기할 때 무조건 랜덤 Feature
- threshold는 가장 좋은 임계값 사용
- 부트스트랩 샘플은 적용되지 않음
- 무작위성을 증가시키면 편향은 늘어나지만 분산은 감소
- 계산비용은 적지만 무작위성때문에 많은 Tree가 필요, RandomForest가 더 선호된다

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

## 6.1.3 Boosting

## 6.1.3.1 Adaboost

- 손실함수가 적은 분류기를 새 멤버로 영입
- 각 멤버가 서로 다른 투표권이 있음, 투표권은 손실함수로 정함
- 데이터마다 가중치가 다름. 기존 위원회가 틀린 데이터의 가중치가 올라감
- 즉 새 멤버가 되려면 기존 위원회가 틀린 문제를 맞히는 모델이 새 멤버가 될 가능성이 높음
- 각 데이터에 대한 가중치를 저장하는 방법은 데사스쿨 참고
- 이진 분류만 가능
- 강한 아웃라이어가 있는 경우 오버피팅

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

모델 = AdaBoostClassifier(기반모델, n_estimators=, learning_rate=)

- learning_rate: 1 이하로 주면 새로운 멤버의 가중치를 강제로 낮춤

## 6.1.3.2 XgBoost

In [ ]:
import xgboost

model_xgb = xgboost.XGBClassifier(n_estimators=1, 
                                  max_depth=, 
                                  objective='binary:logistic', 
                                  learning_rate=, 
                                  subsample=, 
                                  min_child_weight=, 
                                  colsample_bytree=, 
                                  scale_pos_weight=, 
                                  gamma=, 
                                  reg_alpha=, 
                                  reg_lambda=,
                                  random_state=, n_jobs=-1)  # 사이킷런 래퍼 클래스

model_xgb.fit(X데이터, y데이터,
              eval_set=[(검증X, 검증y)],
              eval_metric=평가함수,
              early_stopping_rounds=,
              verbose=False)

- learning_rate: 디폴트 0.1
    - 그레디언트에 곱하는 스텝 사이즈, 보통 0.01 ~ 0.2
- min_child_weight: 디폴트 1
    - 트리에서 추가적으로 가지를 나눌지를 결정하기 위해 필요한 데이터들의 weight 총합
    - 여기서 말하는 weight는 loss값의 2차 미분값을 의미. 
    - 회귀:MSE에서 (y-h_hat)^2의 2차 미분 값은 1, 즉 데이터 개수만큼 합한 것이므로 가지를 나누는데 필요한 총 데이터 개수란 의미
    - binary:logistic(시그모이드)에서는 σ(y_hat)을 2번 미분한 값의 데이터 개수만큼 합
- gamma: 디폴트 0
    - 트리의 리프 노드를 추가적으로 나눌지를 결정할 최소 손실 감소 값
- max_depth: 디폴트 6
    - 보통 3~10
- subsample : 디폴트 1
    - weak learner가 학습에 사용하는 데이터의 샘플링 비율, 보통 0.5 ~ 1
- colsample_bytree: 디폴트 1
    - max_feature와 유사. 트리 생성에 사용하는 feature 샘플링
- reg_lambda: 디폴트 1
    - L2 Regularization
- reg_alpha: 디폴트 0
    - L1 Regularization
- scale_pos_weight: 디폴트 1
    - 비대칭 클래스 데이터 세트의 균형을 유지하기 위한 파라미터
    
**학습 태스크 파라미터**
- objective : 손실 함수 정의
    - binary:logistic, multi:softmax
- eval_metric : 검증에 사용되는 함수, 디폴트 회귀:rmse, 분류:error
    - rmse
    - mae
    - logloss
    - error: Binary classification error rate(0.5 threshold)
    - merror: Multiclass classification error rate
    - mlogloss: Multiclass logloss
    - auc
    
**속성**
- best_ntree_limit
- best_score

## 6.1.3.3 LightGBM
- 데이터가 작을 때 과적합이 발생할 수 있음(약 1만건 이하)
- 균형 트리분할 대신 최대 손실 값을 갖는 리프 노드를 지속적으로 분할하면서 비대칭적
- 카테고리 피처의 자동 변환과 최적 분할
    - 원핫 인코딩을 사용하지 않고도 카테고리형 피처를 최적으로 변환
- GPU 사용 가능
- 연속값 데이터를 이산화해서 메모리 사용 감소
- 분포가 밀집되어 있는 데이터에서 좋음, 고차원 희소 데이터에서는 서포트 벡터 머신이나 신경망보다 저조

In [ ]:
from lightgbm import LGBMClassifier

모델 = LGBClassifier(n_estimators=100, num_leaves=31, min_child_samples=20, max_depth=-1, 
                            learning_rate=0.1, random_state=, n_jobs=-1, early_stopping_rounds=)
모델.fit(categorical_feature=, feature_name=, eval_set=, eval_metric=)

- min_child_samples(min_data_in_leaf): 디폴트 20
    - 리프 노드가 되기 위한 최소한의 데이터 수
- num_leaves: 31
    - 하나의 트리가 가질 수 있는 최대 리프 개수
    - 2의 max_depth 제곱보다 작아야 한다?
- boosting_type : 트리를 생성하는 알고리즘 (디폴트 gbdt)
    - gbdt: 일반적인 그래디언트 부스팅 결정 트리
    - rf: 랜덤 포레스트
    - goss: gradient one side sampling
        - 내부적으로 LightGBM은 첫 번째 1/learning_rate 반복 동안 gbdt 모드를 사용합니다.
- min_split_gain: 디폴트 0.0
    - 추가 리프노드를 만들기 위한 최소 손실 감소량
- max_cat_group: 디폴트 64
    - 카테고리 변수의 범주가 너무 많은 경우 과적합 방지를 위해 그룹으로 묶음
- subsample(bagging_fraction): 디폴트 1.0
    - 데이터 샘플링 비율
- subsample_freq(bagging_freq): 디폴트 0
    - k 번째 이터레이션 마다 배깅
- colsample_bytree(feature_fraction): 디폴트 1.0
    - feature 샘플링 비율
- reg_lambda: 디폴트 0.0
    - 피처 개수가 많을 경우 적용 검토
- reg_alpha: 디폴트 0.0
- max_bin: 디폴트 255
    - 히스토그램 빈 갯수
- device_type: gpu or gpu
- drop_rate: 디폴트 0.1
    - dropout동안 drop되는 이전 트리의 비율
    - boosting_type이 'gbdt'일 경우에는 적용안됨(확인 필요)
- max_drop: 디폴트 50
    - 한 번의 부스팅 반복 동안 드롭된 트리의 최대 수
- is_unbalance: 공식문서 참고
- seed: 
    - porto competition 2등 솔루션에서 사용
- class_weight: 디폴트 None
    - 'balanced'로 하면 손실함수를 계산할 때 unbalanced 클래스에 대해 가중치를 다르게 매김
- subsample for bin
    - number of data that sampled to construct feature discrete bins
    
**학습 태스크 파라미터**
- objective: xgboost와 동일
- metric
    - mae, mse, binary_logloss, multi_logloss
    
**IO parameter**

- max_bin: 연속값을 최대 몇개의 bin으로 나눌 것인지에 대한 것??
- categorical_feature:
- ignore_column:
- save_binary:
    - 데이터셋을 바이너리 파일로 저장해서 다음에 더 빨리 읽음

- **나머지 파라미터는 참고자료 확인**

## fit() 파라미터
- categorical_feature
    - 카테고리 변수를 지정하는데 사용
- feature_name(디폴트:auto)
    - Feature names. If ‘auto’ and data is pandas DataFrame, data columns names are used
- eval_set
    - 검증데이터 전달
    - eval_set=[(train_x, train_y), (valid_x, valid_y)]
- eval_metric

#### lightgbm의 feature importance 시각화

In [ ]:
from lightgbm import plot_importance

fig, ax = plt.subplots(figsize=(10, 12))
plot_importance(lgbm_wrapper, ax=ax)

## 6.2 Regression

# 7. cross validation

## 7.1 1개 모델의 cross validation

In [8]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict


In [ ]:
kfold = KFold(n_splits=, shuffle=True, random_state=)
scores = cross_val_score(모델, X, y, scoring='accuracy', cv=kfold, n_jobs=-1)

print(scores.mean(), scores.std())

# scoring 종류 :
    - 회귀 : 'r2'
    - 분류 : 'accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'average_precision', 'recall_macro',
    - 군집 : 

# 여기서 X, y는 원본 데이터를 훈련, 테스트로 나눈 후에 train, valid로 나누기 전 훈련_X, 훈련_y
# 이 훈련_X, 훈련_y를 다시 train, valid로 나눠서 교차검증이 이루어짐

# cross_val_score에서 폴드를 나눌 때 분류에는 StratifiedKFold, 회귀에는 단순 KFold가 적용됨

# cross_validate 함수를 사용하면 훈련 시간 반환, 여러개의 평가지표 전달 가능(파이썬 라이브러리를 활용한 머신러닝 p.326)

## 7.2 여러개의 모델의 cross validation

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

n_splits = 10
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=)
cv_means = []  # cross_val의 mean
cv_results = []   # cross_val의 result의 리스트들
cv_stds = []
model_names = ['이름1', '이름2', '이름3']
model_list = [모델1, 
              모델2, 
              모델3]

for model in model_list:
    result = cross_val_score(model, X_데이터, Y_데이터, cv=kfold, scoring='accuracy', n_jobs=-1)
    cv_means.append(result.mean())
    cv_stds.append(result.std())
    scores.append(cv_results)

new_models_dataframe = pd.DataFrame({'CV Mean': cv_means, 'Std': cv_stds}, index=model_names)

# 여기서 X, y는 원본 데이터를 훈련, 테스트로 나눈 후에 train, valid로 나누기 전 훈련_X, 훈련_y

In [ ]:
# cv_means 시각화 - barplot 플롯 
plt.subplots(figsize=(12, 8))
sns.barplot('CV Mean', model_names, palette='Set2', data=new_models_dataframe, xerr=1.96*np.array(cv_stds)/np.sqrt(n_splits))
plt.title('Average CV Mean 평가지수 ex)Accuracy')
plt.show()

In [ ]:
# cv_results 시각화 - 박스플롯 
plt.subplots(figsize=(20, 8))
box = pd.DataFrame(cv_results, index=model_names)
box.T.boxplot()
plt.xticks(rotation=45)
plt.show()

# 8. Confusion Matrix
- binary classification 문제

## 8.1 단일 valid_data로 Confusion Matrix 계산

## 8.2 cross validation으로 Confusion Matrix 계산

#### heatmap 으로 시각화

In [ ]:
from sklearn.metrics import confusion_matrix

f, ax = plt.subplots( , , figsize=( , ))

y_pred = cross_val_predict(모델, X, Y, cv=kfold) # 각 폴드가 테스트 세트일 때 예측된 값을 반환
sns.heatmap(confusion_matrix(Y, y_pred), annot=True, fmt='2.0f', ax=ax[ , ])
ax[0, 0].set_title('Matrix for 모델')

# 여기서 X, y는 원본 데이터를 훈련, 테스트로 나눈 후에 train, valid로 나누기 전 훈련_X, 훈련_y

- cross_val_predict를 쓰지 않고 그냥 valid_X, valid_Y만 하면 1개의 폴드만을 이용해서 평가한다


- 정확도(accuracy) : 전체 정확도    
    - (TP + TN) / (TP + TN + FP + FN)
- 정밀도(precision) : 양성이라고 예측한 암환자가 모두 암환자가 확실한가     
    - TP / (TP + FP) 
- 재현율(recall, TPR) : 암환자를 놓치지 않고 모두 찾아낼 수 있는가     
    - TP / (TP + FN)
- 위양성률(fall-out, FPR) : 무고한 사람을 유죄로 몰지 않는가    
    - FP / (FP + TN)
- f1_score: 2 * precision * recall / (precision + recall)

## 8.3 정밀도-재현율 곡선(precision-recall curve)

In [ ]:
from  sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(y_검증데이터, 모델.decision_function(X_검증데이터))

# 판별함수값이 0에 가까운 임계값을 찾음
close_zero = np.argmin(np.abs(thresholds))
plt.plot(precision[close_zero], recall[close_zero], 'o', markersize=10, label='임계값 0', fillstyle='none', c='k', mew=2)

plt.plot(precision, recall, label='정밀도-재현율 곡선')
plt.xlabel('precision')
plt.ylabel('재현율')
plt.legend(loc='best')
plt.show()

- decision_function(X_검증)의 결과에 0이 포함되지 않을 수 있기 때문에 절댓값이 가장 작은 위치의 인덱스를 구하기 위해 argmin

#### 평균정밀도(average precision) - 정밀도-재현율 곡선의 아랫부분 면적

In [ ]:
from sklearn.metrics import average_precision_score

average_precision = average_precision_score(y_검증, 모델.decision_function(X_검증))

## 8.4 ROC-AUC 

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y, 모델.predict_proba)

# 9. Hyper-Parameters Tuning
- Manual tuning 하면서 intuition을 얻는다

## 9.1 GridSearchCV

In [ ]:
model = 모델()

모델_param_grid = {'':, []}

gs모델 = GridSearchCV(model, param_grid=모델_param_grid, cv=kfold, scoring='accuracy', n_jobs=-1, verbose=1)

gs모델.fit(X_훈련, Y_훈련)

## 9.2. Randomized search

## 9.3 Baysian optimization

# 10. learning_curve 시각화
- 훈련데이터 크기가 미치는 영향
- train, valid 곡선이 가까우면 일반화가 잘 된다고 볼 수 있음

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=[0.5, 1.0], cv=None, n_jobs=-1, train_sizes=np.linspace(최소, 1.0, 몇등분)):
    """Generate a simple plot of the test and training learning curve"""
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel('Training examples')
    plt.ylabel('Score')
    
    train_sizes, train_scores, valid_scores = learning_curve(모델.best_estimator_, X, y, cv=kfold, n_jobs=-1, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    valid_scores_mean = np.mean(valid_scores, axis=1)
    valid_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.plot(train_sizes, train_scores_mean, 'o-', color='r', 
             label='Training score')
    plt.plot(train_sizes, valid_scores_mean, 'o-', color='g', 
             label='Cross-validation score')
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, 
                     train_scores_mean + train_scores_std, alpha=0.1, color='r')
    plt.fill_between(train_sizes, valid_scores_mean - valid_scores_std, 
                     valid_scores_mean + valid_scores_std, alpha=0.1, color='g')
    
    plt.legend(loc='best')
    return plt



In [ ]:
plot_learning_curve(모델.best_estimator_, '모델 learning curves', X_훈련, Y_훈련, cv=kfold)

# 11. Feature Importance
- 주로 random forest로 측정
- 모델들을 합쳐서 평균낼 수도 있다
- Decision 기반 모델에서 계산은 feature당 누적 정보획득량으로 계산

## 11.1 Binary classifiacation

In [ ]:
from pandas import Series

feature_importance = model.feature_importances_
Series_feat_imp = Series(feature_importance, index=데이터.columns)
Series_feat_imp

In [ ]:
plt.figure(figsize=(8, 8))
Series_feat_imp.sort_values(ascending=True).plot.barh()
plt.xlabel('Feature importance')
plt.ylabel('Feature')
plt.show()

#### 여러개의 Feature Importance 시각화

In [ ]:
nrows = 
ncols = 
fig, ax = plt.subplots(nrows, ncols, sharex='all', figsize=(15, 15))

names_classifiers = [('모델1이름', 모델1.best_estimator_), 
                     ('모델2이름', 모델2.best_estimator_), 
                     ('모델3이름', 모델3.best_estimator_), 
                     ('모델4이름', 모델4.best_estimator_)]

nclassifier = 0
for row in range(nrows):
    for col in range(ncols):
        name = names_classifiers[nclassifier][0]
        classifier = names_classifiers[nclassifier][1]
        indices = np.argsort(classifier.feature_importances_)[::-1][:40]
        g = sns.barplot(y=X_train.columns[indices][:40], x=classifier.feature_importances_[indices][:40], orient='h', ax=ax[row, col])
        g.set_xlabel('Relative importance', fontsize=12)
        g.set_ylabel('Features', fontsize=12)
        g.tick_params(labelsize=9)
        g.set_title(name + 'feature importance')
        nclassifier += 1

# 12. 모델간의 test데이터 예측결과 상관계수
    - 각 모델들의 best_estimator_를 기준

In [ ]:
test_predict_모델1 = pd.Series(모델1.best_estimator_.predict(test), name='이름1')
test_predict_모델2 = pd.Series(모델2.best_estimator_.predict(test), name='이름2')
test_predict_모델3 = pd.Series(모델3.best_estimator_.predict(test), name='이름3')

ensemble_results = pd.concat([test_predict_모델1, test_predict_모델2, test_predict_모델3], axis=1)

sns.heatmap(ensemble_results.corr(), annot=True)

# 13. Stacking

In [ ]:
def get_stacking_base_datasets(model, X_train, y_train, X_test, n_folds):
    # KFold 생성
    kf = KFold(n_splits=n_folds, shuffle=False)
    
    # 메타 모델 학습 데이터 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train.shape[0], 1))
    test_pred = np.zeros((X_test.shape[0], n_folds))
    print(model.__class__.__name__, 'model 시작')
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train)):
        x_tr = X_train[train_index]
        y_tr = y_train[train_index]
        x_val = x_train[valid_index]
        
        model.fit(x_tr, y_tr)
        train_fold_pred[valid_index, :] = model.predict(x_val).reshape(-1, 1)
        test_pred[:, folder_counter] = model.predict(X_test)
        
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터 생성
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)
    
    return train_fold_pred, test_pred_mean

In [ ]:
모델1_train, 모델1_test = get_stacking_base_datasets(모델1, X_train, y_train, X_test, 폴드수)
모델2_train, 모델2_test = get_stacking_base_datasets(모델2, X_train, y_train, X_test, 폴드수)
모델3_train, 모델3_test = get_stacking_base_datasets(모델3, X_train, y_train, X_test, 폴드수)
모델4_train, 모델4_test = get_stacking_base_datasets(모델4, X_train, y_train, X_test, 폴드수)

In [ ]:
Stack_final_X_train = np.concatenate((모델1_train, 모델2_train, 모델3_train, 모델4_train), axis=1)
Stack_final_X_test = np.concatenate((모델1_test, 모델2_test, 모델3_test, 모델4_test), axis=1)

In [ ]:
메타모델.fit(Stack_final_X_train, y_train)
stack_final = 메타모델.predict(Stack_final_X_test)

#### 스태킹을 클래스로 정의

In [1]:
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=314).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]


                print ("Base model %d: fit %s model | fold %d" % (i+1, str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
                cross_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
                print("cross_score [roc-auc]: %.5f [gini]: %.5f" % (cross_score.mean(), 2*cross_score.mean()-1))
                y_pred = clf.predict_proba(X_holdout)[:,1]                

                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis=1)

        results = cross_val_score(self.stacker, S_train, y, cv=3, scoring='roc_auc')
        # Calculate gini factor as 2 * AUC - 1
        print("Stacker score [gini]: %.5f" % (2 * results.mean() - 1))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict_proba(S_test)[:,1]
        return res

In [ ]:
모델1_params = {}
모델1_params['파라미터'] = 
모델1_params['파라미터'] = 
모델1_params['파라미터'] = 

In [ ]:
# base models
모델1 = 모델(**모델1_params**)

모델2 = 모델(**모델2_params**)

# stacking model
meta_model =  


In [ ]:
stack = Ensemble(n_splits=, stacker=meta_model, base_models = (모델1, 모델2))

In [ ]:
y_prediction = stack.fit_predict(훈련데이터, 훈련타겟, 테스트데이터)

# 14. 기타 캐글 앙상블 테크닉
- porto competition 2등: 스태킹의 바깥 iteration에서 lightGBM의 여러개의 'seed'를 for문 돌려서 또 앙상블